In [1]:
import pandas as pd
import json
import numpy as np
# import cudf as pd
import torch
from torch.nn import DataParallel
from torch.nn.parallel import DistributedDataParallel as DDP
from transformers import BartForConditionalGeneration, BartTokenizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics.pairwise import cosine_similarity
from geopy import distance
import gradio as gr
import matplotlib as plt
from PIL import Image, ImageDraw, ImageFont

2024-04-19 15:15:31.511517: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-19 15:15:31.511547: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-19 15:15:31.512256: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-19 15:15:31.516130: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


___
### Importing, cleaning, and encoding data

In [2]:
business_df = pd.read_json('./Data/yelp_academic_dataset_business.json', lines=True)
business_df = business_df[(business_df['state'] == 'TN') & (business_df['is_open'] == 1)]
business_df['categories'] = business_df['categories'].fillna('')
business_df = business_df[business_df['categories'].str.contains('Restaurants')]

In [3]:
photos_df = pd.read_json('./Data/photos.json', lines=True)
photos_df = photos_df.loc[photos_df['business_id'].isin(business_df['business_id'])]

In [4]:
checkin_df = pd.read_json('./Data/yelp_academic_dataset_checkin.json', lines=True)
checkin_df = checkin_df.loc[checkin_df['business_id'].isin(business_df['business_id'])]

In [5]:
# useful_chunks = []

# for chunk in pd.read_json('./Data/yelp_academic_dataset_review.json', lines=True, chunksize=100000):
#     filtered_chunk = chunk.loc[chunk['business_id'].isin(business_df['business_id'])]
#     useful_chunks.append(filtered_chunk)

# reviews_df = pd.concat(useful_chunks)
# reviews_df.reset_index(drop=True, inplace=True)

In [6]:
tip_df = pd.read_json('./Data/yelp_academic_dataset_tip.json', lines=True)
tip_df = tip_df.loc[tip_df['business_id'].isin(business_df['business_id'])]

In [7]:
summary_reviews_df = pd.read_csv('./Data/nashville_business_reviews_summary.csv', sep='|')

def integrate_reviews(row, sentiment):
    try:
        review = summary_reviews_df.loc[(summary_reviews_df['sentiment'] == sentiment) & (summary_reviews_df['business_id'] == row['business_id'])]['summary'].values[0]
    except:
        review = 'No Reviews'
    return review

business_df['negative_summary'] = business_df.apply(integrate_reviews,axis=1, args=('negative',))
business_df['positive_summary'] = business_df.apply(integrate_reviews,axis=1, args=('positive',))

In [8]:
all_restaurant_types = business_df['categories'].str.split(',').explode().str.strip().value_counts().index
valid_types = all_restaurant_types[:128].tolist()
types_to_remove = ['Restaurants','Event Planning & Services','Caterers','Music Venues','Food Delivery Services','Venues & Event Spaces','Hotels & Travel','Convenience Stores','International Grocery','Performing Arts','Florists','Active Life','Food','Nightlife', 'Arcades', 'Flowers & Gifts','Butcher', 'Jazz & Blues','Party & Event Planning','Dance Clubs', "Arts & Entertainment", "Shopping", "Ethnic Food", "Street Vendors",
    "Karaoke", "Pasta Shops", "Meat Shops", "Pop-Up Restaurants", "Farmers Market","Automotive"]
for type in types_to_remove:
    valid_types.remove(type)

In [9]:
business_df.dropna(subset=['attributes'], inplace=True)
#extract: Outdoor Seating, Alcohol, RestaurantsPriceRange2
business_df['OutdoorSeating'] = business_df['attributes'].apply(lambda x: x.get('OutdoorSeating', None))
business_df['Alcohol'] = business_df['attributes'].apply(lambda x: x.get('Alcohol', None))
business_df['RestaurantsPriceRange2'] = business_df['attributes'].apply(lambda x: x.get('RestaurantsPriceRange2', None))

#fill outdoor seating with false
business_df['OutdoorSeating'].fillna(False, inplace=True)
business_df['OutdoorSeating'].replace({'False': False, 'True': True, 'None': False}, inplace=True)
#fill alcohol with none
business_df['Alcohol'].fillna('none', inplace=True)
business_df['Alcohol'].replace({
                            "u'none'" : 'none',
                            "u'full_bar'" : 'full_bar',
                            "u'beer_and_wine'" : 'beer_and_wine',
                            "'none'" : 'none',
                            "'full_bar'" : 'full_bar',
                            "'beer_and_wine'" : 'beer_and_wine',
                            }, inplace=True)
#fill price range with 2
business_df['RestaurantsPriceRange2'].fillna(2, inplace=True)
business_df['RestaurantsPriceRange2'] = business_df['RestaurantsPriceRange2'].astype(int)

#fill hours with generic hours dict
business_df['hours'].fillna("{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'Wednesday': '0:0-0:0', 'Thursday': '0:0-0:0', 'Friday': '0:0-0:0', 'Saturday': '0:0-0:0', 'Sunday': '0:0-0:0'}", inplace=True)

In [10]:
def encode_top_categories(row, valid_types):
    row_categories = set(row['categories'])
    return [1 if cat in row_categories else 0 for cat in valid_types]


business_df['categories'] = business_df['categories'].str.split(',')
business_df['categories'] = business_df['categories'].apply(lambda x: [str(cat).strip() for cat in x])

In [11]:
mlb = MultiLabelBinarizer(classes=valid_types)

encoded_array = mlb.fit_transform(business_df['categories'])
# Create a DataFrame from the encoded array
encoded_df = pd.DataFrame(encoded_array, columns=mlb.classes_, index=business_df.index)

# Concatenate the original DataFrame with the new encoded DataFrame
business_df = pd.concat([business_df, encoded_df], axis=1)

/home/mattdipinto/miniconda3/envs/rapids-24.02/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['Active Life', 'Adult Entertainment', 'Amateur Sports Teams', 'Amusement Parks', 'Aquariums', 'Arcades', 'Argentine', 'Armenian', 'Art Galleries', 'Arts & Crafts', 'Arts & Entertainment', 'Australian', 'Auto Detailing', 'Auto Glass Services', 'Auto Repair', 'Automotive', 'Axe Throwing', 'Bangladeshi', 'Bartenders', 'Beauty & Spas', 'Bed & Breakfast', 'Beer Tours', 'Beverage Store', 'Bistros', 'Boating', 'Books', 'Bowling', 'Brasseries', 'Brazilian', 'British', 'Butcher', 'Cafeteria', 'Candy Stores', 'Cantonese', 'Car Wash', 'Caterers', 'Cheese Shops', 'Chocolatiers & Shops', 'Cideries', 'Cigar Bars', 'Cinema', 'Colombian', 'Comedy Clubs', 'Community Centers', 'Community Service/Non-Profit', 'Contractors', 'Convenience Stores', 'Cooking Classes', 'Cosmetics & Beauty Supply', 'Country Clubs', 'Couriers & Delivery Services', 'Cuban', 'Cultural Ce

In [12]:
#onehotencode alcohol, outdoor seating, and price using pandas get_dummies
business_df = pd.get_dummies(business_df, columns=['Alcohol', 'OutdoorSeating', 'RestaurantsPriceRange2'], dtype=int)


In [13]:
#scale the review data
scaler = MinMaxScaler()
scaler.fit(business_df[['stars']])
business_df['stars_scaled'] = scaler.transform(business_df[['stars']])

____
### NLP Summarization of the Positive and Negative reviews for each restaurant

In [14]:
# reviews_df.head()

In [15]:
# def encode_review(row):
#     if row['stars'] < 3:
#         return 'negative'
#     elif row['stars'] == 3:
#         return 'neutral'
#     else:
#         return 'positive'

In [16]:
# reviews_df['sentiment'] = reviews_df.apply(encode_review, axis=1)

In [17]:


# # Load pre-trained model and tokenizer
# model_name = 'facebook/bart-large-cnn'  # BART model fine-tuned for CNN/DailyMail summarization
# tokenizer = BartTokenizer.from_pretrained(model_name)
# text_model = BartForConditionalGeneration.from_pretrained(model_name)
# # text_model = DataParallel(text_model)

# # Enable GPU usage
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# text_model.to(device)
# # Example function to summarize text using the BART model
# def summarize_text(text):
#     inputs = tokenizer([text], max_length=1024, return_tensors='pt', truncation=True)
#     inputs = inputs.to(device)
#     summary_ids = text_model.generate(inputs['input_ids'], num_beams=4, max_length=90, early_stopping=True)
#     summary_text = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
#     print(summary_text)
#     return summary_text

# # Group and summarize reviews

# grouped_reviews = reviews_df.loc[reviews_df['sentiment'] != 'neutral'].groupby(['business_id', 'sentiment'])['text'].agg(lambda x: ' '.join(x)).reset_index()

# # Apply the summarization model to the aggregated reviews
# grouped_reviews['summary'] = grouped_reviews['text'].apply(summarize_text)

# grouped_reviews.to_csv('nashville_business_reviews_summary.csv',sep='|', index=False)

In [18]:


# strategy = tf.distribute.MirroredStrategy()

# with strategy.scope():
#     model = Sequential([
#         Embedding(input_dim=10000, output_dim=128, input_length=100),
#         LSTM(128),
#         Dense(1, activation='sigmoid')
#     ])
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# # # Continue with model training


# test_reviews_df = reviews_df.copy()

# def clean_text(text):
#     text = re.sub(r'\W', ' ', str(text))
#     text = text.lower()
#     text = re.sub(r'\s+[a-z]\s+', ' ', text)
#     text = re.sub(r'^[a-z]\s+', ' ', text)
#     text = re.sub(r'\s+', ' ', text, flags=re.I)
#     return text

    
# test_reviews_df['review_clean'] = test_reviews_df['text'].apply(clean_text)

# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('punkt')

# stop_words = set(stopwords.words('english'))
# lemmatizer = WordNetLemmatizer()

# def process_text(text):
#     tokens = nltk.word_tokenize(text)
#     tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
#     return ' '.join(tokens)

# test_reviews_df['review_final'] = test_reviews_df['review_clean'].apply(process_text)

# tokenizer = Tokenizer(num_words=10000)
# tokenizer.fit_on_texts(test_reviews_df['review_final'])
# sequences = tokenizer.texts_to_sequences(test_reviews_df['review_final'])

# max_length = max(len(x) for x in sequences)  # Or choose a fixed length
# review_padded = pad_sequences(sequences, maxlen=max_length, padding='post')

# # # Building the model
# # model = Sequential()
# # model.add(Embedding(input_dim=10000, output_dim=128, input_length=max_length))
# # model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
# # model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

# # # Compile the model
# # model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


____
### Sorting Valid Categories into more general categories

In [19]:
#ChatGPT was kind enough to sort my categories

american_cuisine = [
    "American (Traditional)", "American (New)", "Burgers", "Barbeque",
    "Southern", "Steakhouses", "Comfort Food", "Cajun/Creole", "Hot Dogs", 
    "New Mexican Cuisine"
]

international_cuisine = [
    "Mexican", "Tex-Mex", "Italian", "Chinese", "Japanese", "Sushi Bars",
    "Asian Fusion", "Mediterranean", "Greek", "Thai", "Latin American",
    "Middle Eastern", "Indian", "Vietnamese", "French", "Korean", "Spanish",
    "Turkish", "Caribbean", "Ramen", "Salvadoran", "Poke", "Hawaiian",
    "Laotian", "Halal", "Ethiopian", "African"
]

fast_food_casual = [
    "Fast Food", "Sandwiches", "Pizza", "Chicken Wings", "Tacos", "Diners",
    "Food Trucks", "Hot Dogs", "Fish & Chips", "Donuts", "Waffles", "Acai Bowls",
    "Wraps", "Cheesesteaks", "Food Court"
]

bars_nightlife = [
    "Bars", "Cocktail Bars", "Sports Bars", "Pubs", "Lounges", "Dive Bars",
    "Wine Bars", "Beer Bar", "Tapas/Small Plates", "Gastropubs", "Breweries",
    "Brewpubs", "Beer Gardens", "Whiskey Bars", "Hookah Bars"
]

health_specialty_foods = [
    "Salad", "Vegetarian", "Vegan", "Gluten-Free", "Juice Bars & Smoothies",
    "Health Markets"
]

beverages = [
    "Coffee & Tea", "Specialty Food", "Wine & Spirits", "Beer", "Coffee Roasteries",
    "Bubble Tea"
]

desserts_bakeries = [
    "Desserts", "Ice Cream & Frozen Yogurt", "Bakeries", "Creperies"
]

cultural_local_flavors = [
    "Local Flavor", "Soul Food"
]


In [20]:
list_of_cats = {
    'American':american_cuisine,
    'International':international_cuisine,
    'Health Food':health_specialty_foods,
    'Local and Cultural':cultural_local_flavors,
    'Fast Food':fast_food_casual,
    'Coffee and Beverages':beverages,
    'Dessert':desserts_bakeries,
    'Bars and Nightlife':bars_nightlife,
}

price_dict = {
    '$':'RestaurantsPriceRange2_1',
    '$$':'RestaurantsPriceRange2_2',
    '$$$':'RestaurantsPriceRange2_3',
    '$$$$':'RestaurantsPriceRange2_4'
}

bar_dict = {
"Beer and Wine":'Alcohol_beer_and_wine',
"Full Bar":'Alcohol_full_bar',
"None":'Alcohol_none'
}

____
### Establishing variable weights for restaurant recommendations through cosine similarity

In [21]:
X_list = business_df.columns[16:]
X = business_df[X_list]
y = business_df['name']

In [22]:
user_dict = {}
for column in X_list:
    user_dict[column] = 0

In [23]:
user_vector = pd.Series(user_dict).values.reshape(1,-1)
user_vector

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

____
### Creating the no-image-found Image

In [24]:
# Settings for the image
width, height = 200, 200  # Dimensions of the image
background_color = 'grey'  # Background color
text = 'No Image Found'  # Text to display
font_color = 'white'  # Color of the text

# Create a new image with the background color
no_photos_img = Image.new('RGB', (width, height), color = background_color)

# Get a drawing context
draw = ImageDraw.Draw(no_photos_img)

# Specify a font (optional: specify a path to a .ttf file if a specific font is desired)
# font = ImageFont.truetype('arial.ttf', 36)  # Example for custom font
# Or use a default font
font = ImageFont.load_default()

# Calculate text width and height
text_width, text_height = 75, 75

# Calculate position at center
x = (width - text_width) / 2
y = (height - text_height) / 2

# Draw text on image
draw.text((x, y), text, font=font, fill=font_color)

# Show the image in a viewer
# no_photos_img.show()


In [25]:
# strategy = tf.distribute.MirroredStrategy()

# with strategy.scope():
#     # Initialize the model
#     model = Sequential()

#     # Adding layers to the model
#     model.add(Dense(len(X_list), activation='relu', input_dim=X_train.shape[1]))
#     model.add(Dropout(0.5))
#     model.add(Dense(64, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(32, activation='relu'))
#     model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

#     # Compile the model
#     model.compile(optimizer='adam',
#                   loss='categorical',
#                   metrics=['mean_squared_error', 'mean_absolute_error'])

# history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.2)

# loss, mse, mae = model.evaluate(X_test, y_test)
# print(f"Test MSE: {mse}, Test MAE: {mae}")
# import matplotlib.pyplot as plt

# def plot_history(history):
#     # Plotting training & validation loss values
#     plt.figure(figsize=(12, 6))
#     plt.subplot(1, 2, 1)
#     plt.plot(history.history['loss'])
#     plt.plot(history.history['val_loss'])
#     plt.title('Model Loss')
#     plt.ylabel('Loss')
#     plt.xlabel('Epoch')
#     plt.legend(['Train', 'Test'], loc='upper left')

#     # Plotting training & validation accuracy values
#     plt.subplot(1, 2, 2)
#     plt.plot(history.history['mean_squared_error'])
#     plt.plot(history.history['val_mean_squared_error'])
#     plt.title('Model Mean Squared Error')
#     plt.ylabel('Mean Squared Error')
#     plt.xlabel('Epoch')
#     plt.legend(['Train', 'Test'], loc='upper left')

#     plt.show()

# plot_history(history)
# # Early stopping callback
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# # Include in the fit function
# history_e_s = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

# plot_history(history_e_s)
# loss, mse, mae = model.evaluate(X_test, y_test)
# print(f"Test MSE: {mse}, Test MAE: {mae}")
# user_dict = {}
# for item in X_list:
#     user_dict[item] = 0

___
### Making a UI

* from the AirBNB selection, the function will receive a lat-long. That latlong will calculate the distance to every restaurant 

The user will select:

* distance they're willing to travel (slider)
* price preferences
* food category or specific food
* indoor/outdoor
* do they want drinks

In [26]:
weight_factor = 2.25
rating_factor = 1.25
weights_array = np.ones_like(user_vector)
weights_array[:,:-10] = weight_factor
weights_array[:,-1] = rating_factor

In [28]:
def restaurant_distances(row,lat,long):
    return round(distance.distance((lat,long),(row['latitude'],row['longitude'])).miles,1)

def populate_range_df(airbnb_lat, airbnb_long):
    business_df['airbnb_range'] = business_df.apply(restaurant_distances, axis=1, args=(airbnb_lat, airbnb_long))

def reset_user():
    global user_dict
    user_dict = {}
    for column in X_list:
        user_dict[column] = 0
    user_dict['stars_scaled'] = rating_factor
    print(pd.Series(user_dict).values.reshape(1,-1))

def calculate_best_restaurant(choice, option, price_range, indoor_outdoor, drinks, distance):
    user_dict['stars_scaled'] = rating_factor
    #first - filter by rules-type things (distance)
    search_df = business_df.loc[business_df['airbnb_range'] <= distance].copy()

    #second - determine if the user choice was a single cuisine or a category
    if choice == 'Category':
        for cuisine in list_of_cats[option]:
            user_dict[cuisine] = weight_factor
    else:
        user_dict[option] = weight_factor

    #third - build the users other choices
    for price in price_dict.keys():
        if price in price_range:
            user_dict[price_dict[price]] = 1
    
    if indoor_outdoor == 'Outdoor Seating':
        user_dict['OutdoorSeating_True'] = 1

    if 'None' in drinks:
        user_dict['Alcohol_none'] = 1
    elif "Doesn't Matter" in drinks:
        user_dict['Alcohol_beer_and_wine'] = 1
        user_dict['Alcohol_full_bar'] = 1
        user_dict['Alcohol_none'] = 1
    else:
        for bar_type in drinks:
            user_dict[bar_dict[bar_type]] = 1
        
    #fourth - calculate cosine similarities
    similarities = cosine_similarity(search_df[X_list].values*weights_array, pd.Series(user_dict).values.reshape(1,-1))
    print(pd.Series(user_dict).values.reshape(1,-1))
    search_df['Match'] = similarities
    search_df.sort_values(by='Match', inplace=True, ascending=False)

    business_id1, business_id2, business_id3 = search_df.head(3)['business_id'].values.tolist()
    search_df.set_index('business_id', drop=True, inplace=True)
    search_df.rename(columns={'name':'english_name'}, inplace=True)
    business1 = search_df.loc[business_id1]
    business2 = search_df.loc[business_id2]
    business3 = search_df.loc[business_id3]

    business_1_description = f"{business1.english_name} is located {business1.airbnb_range} miles from your airbnb.\n\nThe positive reviews say: {business1.positive_summary}\n\nThe negative reviews say: {business1.negative_summary}"
    try:
        b1_img = Image.open(f"./Data/photos/{photos_df.loc[photos_df['business_id'] == business_id1]['photo_id'].values[0]}.jpg")
    except:
        b1_img = no_photos_img

    business_2_description = f"{business2.english_name} is located {business2.airbnb_range} miles from your airbnb.\n\nThe positive reviews say: {business2.positive_summary}\n\nThe negative reviews say: {business2.negative_summary}"
    try:
        b2_img = Image.open(f"./Data/photos/{photos_df.loc[photos_df['business_id'] == business_id2]['photo_id'].values[0]}.jpg")
    except:
        b2_img = no_photos_img

    business_3_description = f"{business3.english_name} is located {business3.airbnb_range} miles from your airbnb.\n\nThe positive reviews say: {business3.positive_summary}\n\nThe negative reviews say: {business3.negative_summary}"
    try:
        b3_img = Image.open(f"./Data/photos/{photos_df.loc[photos_df['business_id'] == business_id3]['photo_id'].values[0]}.jpg")
    except:
        b3_img = no_photos_img
    return business_1_description, b1_img, business_2_description, b2_img, business_3_description, b3_img

def update_options(choice):
    if choice == "Specific Food":
        return gr.Dropdown(choices=valid_types)
    elif choice == "Category":
        return gr.Dropdown(choices=list(list_of_cats.keys()))
    return []  # return an empty list if no choice is made


In [29]:

# populate_range_df(36.269593,-87.058943)
reset_user()

[[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   1.25]]


In [31]:
theme1 = gr.themes.Soft(
    primary_hue="sky",
    secondary_hue="red",
    radius_size="lg",
)

theme2 = gr.themes.Glass()

with gr.Blocks(title='Restaurant Recommendations') as Restaurants:
    with gr.Row(variant='compact'):
            import_airbnb_button = gr.Button('Import BnB')
    with gr.Row():  
        with gr.Row():
            choice = gr.Radio(["Specific Food", "Category"], label="What would you like to do?")
            option = gr.Dropdown(['Select Specific Food or Category'],label="Choose an option", value='Select Specific Food or Category', scale=2)
        with gr.Row():
            price_range = gr.CheckboxGroup(['$','$$','$$$','$$$$'], label="Price Range", info="What Price Ranges are you feeling?")
            indoor_outdoor = gr.Radio(['Indoor Seating', 'Outdoor Seating'], label='Indoor or Outdoor seating?',)
    with gr.Row():
        drinks = gr.CheckboxGroup(["Doesn't Matter","Beer and Wine","Full Bar","None"], label="Alcohol Available?", info="Select what type of drinks you would like available.")
        distance_slider = gr.Slider(value=5, minimum=0.1, maximum=30, label='Max distance from your AirBNB', interactive=True)
    with gr.Row():   
        submit_btn = gr.Button("Submit")
        reset_button = gr.Button('Reset User Preferences')

    with gr.Row():
        output1 = gr.Textbox(scale=3, visible=True, show_label=False, interactive=False)
        photo1 = gr.Image(scale=1, visible=True,show_label=False)
    with gr.Row():
        output2 = gr.Textbox(scale=3, visible=True, show_label=False, interactive=False)
        photo2 = gr.Image(scale=1, visible=True,show_label=False)
    with gr.Row():
        output3 = gr.Textbox(scale=3, visible=True, show_label=False, interactive=False)
        photo3 = gr.Image(scale=1, visible=True,show_label=False)

    choice.input(update_options, inputs=choice, outputs=option)
    submit_btn.click(fn=calculate_best_restaurant, inputs=[choice, option, price_range, indoor_outdoor, drinks, distance_slider], outputs=[output1, photo1, output2, photo2, output3,photo3])
    reset_button.click(fn=reset_user)
    import_airbnb_button.click(fn=populate_range_df)

interface = gr.TabbedInterface([Restaurants], ["Tabby McTabface"], theme=theme1)
interface.launch(share=True)

/home/mattdipinto/miniconda3/envs/rapids-24.02/lib/python3.10/site-packages/gradio/utils.py:924: UserWarning: Expected 2 arguments for function <function populate_range_df at 0x7728bf5a2b90>, received 0.
  warnings.warn(
/home/mattdipinto/miniconda3/envs/rapids-24.02/lib/python3.10/site-packages/gradio/utils.py:928: UserWarning: Expected at least 2 arguments for function <function populate_range_df at 0x7728bf5a2b90>, received 0.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://71ce15013f2fda6f72.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/home/mattdipinto/miniconda3/envs/rapids-24.02/lib/python3.10/site-packages/gradio/helpers.py:940: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
Traceback (most recent call last):
  File "/home/mattdipinto/miniconda3/envs/rapids-24.02/lib/python3.10/site-packages/gradio/queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
  File "/home/mattdipinto/miniconda3/envs/rapids-24.02/lib/python3.10/site-packages/gradio/route_utils.py", line 261, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/mattdipinto/miniconda3/envs/rapids-24.02/lib/python3.10/site-packages/gradio/blocks.py", line 1786, in process_api
    result = await self.call_function(
  File "/home/mattdipinto/miniconda3/envs/rapids-24.02/lib/python3.10/site-packages/gradio/blocks.py", line 1338, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/home/mattdipinto/miniconda

[[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   1.25]]
[[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.  

In [ ]:
import gradio as gr

In [ ]:
gr.themes.builder()

In [ ]:
def update_visibility(radial_input):
    if radial_input == 'No':
        return gr.Textbox(visible=False), gr.Button(visible=False)
    else:
        return gr.Textbox(visible=True), gr.Button(visible=True)

def update_itinerary(query):
    response = "Whatever the model said"
    return response, gr.Radio(['Yes','No'], value=None,label="Was this better?"), gr.Textbox(value="", visible=False), gr.Button(visible=False)

theme = gr.themes.Soft()

with gr.Blocks(theme=theme) as demo:
    with gr.Row():
        click_me = gr.Radio(['Yes','No'])
        results_box = gr.Textbox(interactive=False)
    with gr.Row():
        text_box = gr.Textbox(visible=False)
        submit_button2 = gr.Button(visible=False)


    click_me.input(update_visibility,inputs=click_me, outputs=[text_box,submit_button2])
    submit_button2.click(update_itinerary, inputs=text_box, outputs=[results_box, click_me, text_box, submit_button2])

demo.launch()